In [ ]:
!pip install python-pptx 

In [ ]:
from typing import Final
import boto3, os, pptx, json
polly = boto3.client('polly')
pptx_file_path:Final = './test.pptx' 
voice:Final = 'Takumi'

In [ ]:
mp3_dir: Final[str] = 'mp3'
!if [ -d ./{mp3_dir} ]; then rm -rf ./{mp3_dir}/;fi
!mkdir ./{mp3_dir}/

In [ ]:
def generate_ssml(text,speed=100):
    ssml = f'<speak><prosody rate="{speed}%">{text}</prosody></speak>'
    return ssml

In [ ]:
def generate_mp3(ssml,mp3_dir,mp3_name,voice):
    response = polly.synthesize_speech(
        Engine='neural',
        Text = ssml,
        OutputFormat = "mp3", 
        TextType='ssml',
        VoiceId = voice,
    )
    with open(os.path.join(mp3_dir,mp3_name), "wb") as f:
        f.write(response['AudioStream'].read())

In [ ]:
prs = pptx.Presentation(pptx_file_path)
memo_list = []
for i, sld in enumerate(prs.slides):
    text = sld.notes_slide.notes_text_frame.text
    memo_list.append({
        'index' : str(i+1).zfill(3),
        'ssml' : generate_ssml(sld.notes_slide.notes_text_frame.text,150)
    })

In [ ]:
# ここでSSMLをチェック＆修正する
print(
    json.dumps(
        memo_list, indent=4
    ).encode().decode('unicode-escape')
)

In [ ]:
for memo in memo_list:
    generate_mp3(memo['ssml'], mp3_dir, memo['index']+'.mp3', voice)

In [ ]:
!zip -r mp3.zip ./{mp3_dir}